# Using  EAGO with a script-defined problem:  
### *Kinetic parameter estimation with explicit Euler integration*

[Matthew Wilhelm](https://psor.uconn.edu/person/matthew-wilhelm/)  
Department of Chemical and Biomolecular Engineering, University of Connecticut

[Robert Gottlieb](https://psor.uconn.edu/person/robert-gottlieb/)  
Department of Chemical and Biomolecular Engineering, University of Connecticut

Consider the kinetic parameter estimation problem described in [1,2,3]. It consists of a system of ODEs that describe the concentration of the involved species after initial laser flash pyrolysis given below:

$
\begin{align}
    \frac{dx_A}{dt} &= k_1 x_Z x_Y - c_{O_2} (k_{2f}+k_{3f})x_A + \frac{k_{2f}}{K_2}x_D + \frac{k_{3f}}{K_3}x_B - k_5 x_A^2 \\
     \frac{dx_B}{dt} &= c_{O_2}k_{3f}x_A - \left(\frac{k_{3f}}{K_3}+k_4\right)x_B,\\
     \frac{dx_D}{dt} &= c_{O_2}k_{2f}x_A - \frac{k_{2f}}{K_2}x_D, \\
     \frac{dx_Y}{dt} &= - k_{1s} x_Z x_Y, \\
     \frac{dx_Z}{dt} &= - k_1 x_Z x_Y, \\
x_A(0) &= 0,\;  x_B(0) = 0,\; x_D(0) = 0,\; x_Y(0) = 0.4,\; x_Z(0) = 140     
\end{align}
$

where $x_j$ is the concentration of species $j \in \{A,B,D,Y,Z\}$. The constants are then given by $T = 273$, $K_2 = 46\exp(6500/T-18)$, $K_3=2K_2$, $k_1 = 53$, $k_{1s} = k_1 \times 10^{-6}$, $k_5 = 1.2\times 10^{-3}$, and $c_{O_2} = 2\times 10^{-3}$. 

One seeks to determine the reaction rate constant from measured intensity data by minimizing the sum-squared-error. A known dependency of intensity on concentrations exists and is given by $I = x_A + \frac{2}{21}x_B + \frac{2}{21}x_D$. The reaction rate constants are $k_{2f} \in [10,1200]$, $k_{3f} \in [10,1200]$, and $k_4 \in [0.001,40]$ and form the decision space vector $\mathbf{p} = (k_{2f},k_{3f},k_4)$. In the below example, we'll discretize the ODE system via an explicit Euler method taking $\Delta t = 0.01$ and formulate an optimization problem which we'll then solve using EAGO's **script_solve** function.

For reference, the explicit Euler discretization is given by:

$
\begin{align}
x_A^{i+1} &= x_A^i + \Delta t \left(k_1 x_Y^{i} x_Z^{i} - c_{O2}(k_{2f}+k_{3f})x_A^i + \frac{k_{2f}}{K_2}x^i_D + \frac{k_{3f}}{K_3}x^i_B - k_5 (x_A^i)^2 \right) \\
x_B^{i+1} &= x_B^i + \Delta t \left(k_{3f}C_{O2}x_A^i - \left(\frac{k_{3f}}{K_3} + k_4\right)x_B^i\right) \\
x_D^{i+1} &= x_D^i + \Delta t \left(k_{2f}C_{O2}x_A^i - \frac{k_{2f}}{K_2} x_D^i\right) \\
x_Y^{i+1} &= x_Y^i + \Delta t \left(-k_{1s} x_Y^i x_Z^i \right) \\
x_Z^{i+1} &= x_Z^i + \Delta t \left(k_{1} x_Y^i x_Z^i\right)
\end{align}
$

### Load data
We now load the data from the "kinetic_intensity_data.csv" file and bounds from the "implicit_variable_bounds.csv" file.

In [17]:
using EAGO, JuMP, DataFrames, CSV

data = CSV.read("kinetic_intensity_data.csv", DataFrame)
bounds = CSV.read("implicit_variable_bounds.csv", DataFrame)

pL = [10.0, 10.0, 0.001]
pU = [1200.0, 1200.0, 40.0];

### Define explicit Euler integration function

In [18]:
# Define the explicit Euler integration scheme
function explicit_euler_integration(p::T...) where {T}
    x = zeros(T, 1005)
    x[4] = 0.4
    x[5] = 140

    # sets known parameter values
    Temp = 273.0
    K2 = 46.0*exp(6500.0/Temp - 18.0)
    K3 = 2.0*K2
    k1 = 53.0
    k1s = k1*10^(-6)
    k5 = 0.0012
    cO2 = 0.002

    h = 0.01
    # offset by 1, since the initial condition is x[1:5]
    for i = 1:200
        term1 = k1*x[5i-1]*x[5i] - cO2*(p[1] + p[2])*x[5i-4]
        term2 = p[1]*x[5i-2]/K2 + p[2]*x[5i-3]/K3 - k5*x[5i-4]^2
        x[5i+1] = x[5i-4] + h*(term1 + term2)
        x[5i+2] = x[5i-3] + h*(p[2]*cO2*x[5i-4] - (p[2]/K3 + p[3])*x[5i-3])
        x[5i+3] = x[5i-2] + h*(p[1]*cO2*x[5i-4] - p[1]*x[5i-2]/K2)
        x[5i+4] = x[5i-1] + h*(-k1s*x[5i-1]*x[5i])
        x[5i+5] = x[5i] + h*(-k1*x[5i-1]*x[5i])
    end
    return x
end;

### Define the objective function

In [19]:
# Defines function for intensity
intensity(xA, xB, xD) = xA + (2/21)*xB + (2/21)*xD

# Defines the objective: integrates the ODEs and calculates SSE
function objective(p::T...) where {T}
    x = explicit_euler_integration(p...)
    SSE = zero(T)
    for i = 1:200
        SSE += (intensity(x[5i+1], x[5i+2], x[5i+3]) - data[!,:intensity][i])^2
    end
    return SSE
end;

### Solve the problem and get outputs

In [20]:
# Create model and add variables)
model = Model(optimizer_with_attributes(EAGO.Optimizer, "relative_tolerance" => 0.05, "unbounded_check" => false))

@variable(model, pL[i] <= p[i=1:3] <= pU[i])

# Register objective, add objective function, and optimize
fobj(p...) = objective(p...)
@operator(model, op_fobj, 3, fobj)
@objective(model, Min, op_fobj(p...))
JuMP.optimize!(model)

---------------------------------------------------------------------------------------------------------------------------------
|  Iteration #  |     Nodes     |  Lower Bound  |  Upper Bound  |      Gap      |     Ratio     |     Timer     |   Time Left   |
---------------------------------------------------------------------------------------------------------------------------------
|          1000 |           867 |     8.983E+03 |     9.628E+03 |     6.442E+02 |     6.692E-02 |         20.88 |       3579.12 |
|          2000 |          1863 |     9.055E+03 |     9.628E+03 |     5.722E+02 |     5.943E-02 |         32.56 |       3567.44 |
|          3000 |          2861 |     9.092E+03 |     9.628E+03 |     5.356E+02 |     5.563E-02 |         43.65 |       3556.35 |
|          4000 |          3857 |     9.115E+03 |     9.628E+03 |     5.122E+02 |     5.321E-02 |         54.31 |       3545.69 |
|          5000 |          4857 |     9.132E+03 |     9.628E+03 |     4.954E+02 |     5.14

Note that the EAGO solver defaults to refine interval bounds using subgradients as described in [4,5] for it's basic **JuMP.optimize!** routine.

### References
1. J. W. Taylor, et al., Direct measurement of the fast, reversible addition of oxygen to cyclohexadienyl radicals in nonpolar solvents, *J. Phys. Chem. A.*, **2004**, 108, 7193–7203.
2. A. B. Singer, et al., Global dynamic optimization for parameter estimation in chemical kinetics, *J. Phys. Chem A.*, **2006**, 110, 971–976.
3. Mitsos, A. Chachuat, B., & Barton, P.I., McCormick-based relaxations of algorithms, *SIAM Journal on Optimization*, *SIAM*, **2009**, 20(2), 573-601.
4. Stuber, M.D., Scott, J.K., & Barton, P.I.: Convex and concave relaxations of implicit functions. *Optim. Methods Softw.*, **2015** 30(3), 424–460.
5. Najman, J., Mitsos, A., Tighter McCormick relaxations through subgradient propagation *arXiv preprint* arXiv:1710.09188